In [1]:
import auth
import requests
import base64
import json
import pandas as pd
flickr = auth.connect()

In [2]:
df = pd.read_csv('../us_cities.csv')


top_cities = ["New York", "Los Angeles", "Las Vegas", "Orlando", "Miami", "San Francisco"]

df_top = df[df['CITY'].isin(top_cities)]
df_top = df_top.reset_index()  

In [3]:
cities= {
   "New York":{
      "latitude":40.730610,
      "longitude":-73.935242
   },
   "Los Angeles":{
      "latitude":34.052235,
      "longitude":-118.243683
   },
   "Las Vegas":{
      "latitude":36.188110,
      "longitude":-115.176468
   },
   "Orlando":{
      "latitude":28.538336,
      "longitude":-81.379234
   },
   "Miami":{
      "latitude":25.761681,
      "longitude":-80.191788
   },
   "San Francisco":{
      "latitude":37.773972,
      "longitude":-122.431297
   }
}


In [4]:
# cities= {
#    "New York":{
#       "latitude":40.730610,
#       "longitude":-73.935242
#    }
# }

Fields needed for each photo:
id: unique ID of the uploaded photo (pid).
owner-id: unique user ID of the person who uploaded the photo (up).
longitude: (geotag information) x coordinate (xp).
latitude: (geotag information) y coordinate (yp).
dates-taken: date and time when the photo was taken (tp).
dates-posted: date and time when the photo was uploaded.

In [5]:
def getlonglat(row):
    res = flickr.photos.geo.getLocation(photo_id=str(row['id']))
    
    location = res['photo']['location']
    row['latitude'] = location['latitude']
    row['longitude'] = location['longitude']
    return row


In [ ]:
for k,v in cities.items():
    data=[]
    pages = 8
    city = k
    long = v['longitude']
    lat = v['latitude']
    filename= city+'-photos.csv'

    for p in range(1, pages+1):
        
    #     photos = flickr.photos.search(lat=lat, lon=long, accuracy=11,min_date_taken='2010-01-01', max_date_taken='2023-12-31', radius = 20, radius_units='mi', extras=["date_upload", "date_taken", "owner_name", "geo"])
        photos = flickr.photos.search(lat=lat, lon=long, accuracy=11, radius = 20, radius_units='mi',min_date_taken='2010-01-01', max_date_taken='2023-12-31', extras=["date", "date_upload", "date_taken", "owner_name"], page=p)

        df_photos = pd.DataFrame(photos['photos']['photo'])
        df_photos = df_photos.drop(['secret', 'farm', 'server', 'ispublic', 'isfriend', 'isfamily'], axis=1)
        df_photos = df_photos.apply(getlonglat, axis=1)
        df_photos = df_photos.drop(df_photos.columns[0], axis=1)
        df_photos = df_photos.drop_duplicates()
        data.append(df_photos)
    data = pd.concat(data)
    data.to_csv(filename)


 

In [ ]:
df1= pd.read_csv('New York-photos.csv')

In [ ]:
df1

In [ ]:
photos_by_user = df1.groupby("owner")['id'].count().sort_values(ascending= False)

In [ ]:
photos_by_user